In [1]:
import os
import nltk
import os
import nltk
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())
import importlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from transformers import BertForSequenceClassification
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())
sentences = []
labels = []

with open("amazon_cells_labelled.txt", "r") as file:
    for line in file:
        # Strip whitespace, split the sentence and label
        line = line.strip()
        if line:
            line = line.replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '')
            line = line.replace('[^\w\s]','')
            line = line.replace('\d', '') 
            line = line.replace('\t', '') 
            labels.append(int(line[-1]))
            sentences.append(line[:len(line)-1])
    file.close()



/Lab1


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-14 14:21:51.206005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744633311.230456   53465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744633311.237745   53465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744633311.256106   53465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17446

/Lab1


In [2]:
from transformers import BertTokenizer

# Load the tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',num_labels=2)

# Tokenize the sentences
encoding = tokenizer(
    sentences,                      # List of sentences
    add_special_tokens=True,        # Add [CLS] and [SEP] tokens
    max_length=128,                 # Truncate/pad to this max length
    padding='max_length',           # Pad all sentences to the same length
    truncation=True,                # Truncate sentences longer than max_length
    return_tensors="pt"             # Return PyTorch tensors
)

# Extract input_ids and attention_mask
input_ids = encoding['input_ids']            # Shape: [num_sentences, max_length]
attention_mask = encoding['attention_mask']  # Shape: [num_sentences, max_length]

In [3]:
import torch

# Convert labels to a PyTorch tensor
labels = torch.tensor(labels)  # Shape: [num_sentences]

In [4]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

# Create a dataset
dataset = TensorDataset(input_ids, attention_mask, labels)
print(len(dataset))

# Specify dataset sizes (e.g., 70% train, 15% validation, 15% test)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Randomly split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
batch_s = 16
train_loader = DataLoader(train_dataset, batch_size=batch_s, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_s)
test_loader = DataLoader(test_dataset, batch_size=batch_s)

1000


In [5]:
from transformers import BertForSequenceClassification
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizer
from transformers import BertModel
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
print(torch.cuda.is_available())  # Returns True if a GPU is available
# Specify the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move your model to the device
model = model.to(device)

# Move your data tensors to the device


True


In [7]:
epochs = 7
learning_rate = 1e-5
optimizer = optim.AdamW(model.parameters(),lr=learning_rate,weight_decay=0)
loss_fn = torch.nn.CrossEntropyLoss()
best_loss = 10000

for epoch in range(epochs):
    avg_train_loss = 0
    avg_val_loss = 0

    model.train()
    for batch in train_loader:
        input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
        attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
        labels = batch[2].to(device)              # Move labels to GPU/CPU
        output = model(input_ids,attention_mask=attention_mask)
        output = output.logits

        loss = loss_fn(output,labels)
        avg_train_loss += loss.item()

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        

        

    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
            attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
            labels = batch[2].to(device)              # Move labels to GPU/CPU
            output = model(input_ids,attention_mask=attention_mask)
            output = output.logits
            loss = loss_fn(output,labels)
            avg_val_loss += loss.item()
    torch.cuda.empty_cache()

    avg_train_loss = avg_train_loss/len(train_loader)
    avg_val_loss = avg_val_loss/len(val_loader)
    del output, loss

    
    print("Epoch: ",epoch,"Train loss: ",avg_train_loss,"Val loss: ", avg_val_loss)

Epoch:  0 Train loss:  0.6008738137104295 Val loss:  0.4376864582300186
Epoch:  1 Train loss:  0.2947336401451718 Val loss:  0.2431981161236763
Epoch:  2 Train loss:  0.1429926950315183 Val loss:  0.23353499844670295
Epoch:  3 Train loss:  0.0711672600274059 Val loss:  0.22837235275655984
Epoch:  4 Train loss:  0.06539744985374538 Val loss:  0.23265056516975163
Epoch:  5 Train loss:  0.04597081049260768 Val loss:  0.25546308448538185
Epoch:  6 Train loss:  0.050173930887302216 Val loss:  0.2952569654211402


In [8]:

total = 0
correct = 0
model.eval()
for batch in val_loader:
    input_ids = batch[0].to(device)           # Move input_ids to GPU/CPU
    attention_mask = batch[1].to(device)      # Move attention_mask to GPU/CPU
    labels = batch[2].to(device)              # Move labels to GPU/CPU
    output = model(input_ids,attention_mask=attention_mask)
    output = output.logits
    for i in range(len(output)):
        o = torch.argmax(output[i]).item()
        label = labels[i].item()
        total += 1
        if  o == label:
            correct += 1
print(correct/total)
    

0.9
